# Registro de detecciones

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../videoanalytics/src")
import videoanalytics
videoanalytics.__version__

'0.0.2'

In [2]:
from videoanalytics.pipeline import Pipeline
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter

from videoanalytics.pipeline.sinks.object_detection import DetectionsAnnotator, DetectionsCSVWriter
from videoanalytics.pipeline.sinks.object_detection.yolo4 import YOLOv4DetectorTF

def make_detection_pipeline(input_video_filename,
                            output_video_filename,
                            detector_model_weights_filename,
                            detector_model_classes_filename,
                            start_frame=0,
                            max_frames=None):
    context = {}
    pipeline = Pipeline()

    pipeline.add_component( VideoReader( "input",context,
                     video_path=input_video_filename,
                     start_frame=start_frame,
                     max_frames=max_frames))    
    
    pipeline.add_component( YOLOv4DetectorTF("detector",
                                             context,
                                             weights_filename=detector_model_weights_filename) )
    
    pipeline.add_component( DetectionsAnnotator("annotator",
                                                context,
                                                class_names_filename=detector_model_classes_filename,
                                                show_label=True) )
    
    pipeline.add_component(VideoWriter("writer",context,filename=output_video_filename))
    
    
    pipeline.set_connections([
        ("input", "detector"),
        #("detector", "det_csv_writer"),
        ("detector", "annotator"),
        ("annotator", "writer")
    ])    
    
    return context,pipeline

/home/nhorro/anaconda3/envs/ai-fisheries/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nhorro/anaconda3/envs/ai-fisheries/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nhorro/anaconda3/envs/ai-fisheries/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/n

In [3]:
WORKSPACE_PATH="../"
WORKSPACE_DATA_PATH=WORKSPACE_PATH+"/data/"
INPUT_VIDEO_PATH=WORKSPACE_DATA_PATH+"media/videos-youtube/"
!ls {INPUT_VIDEO_PATH}

'buque EL MARISCO II ,proceso de captura merluz.mp4'
 cut
'LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4'
'Maniobra de pesca de merluza en mares argentino.mp4'
 maniobr.mp4
'Pesca artesanal de anchoita en mar del plat.mp4'
'pesca de altura en el mar argentino,pesquero.mp4'
'pesca de langostino,buque Alve.mp4'
'Pesca de langostino en aguas nacionale.mp4'
'pesca de langostino en rawson chubut con barco combenciona [camara-fija].mp4'
'PESCA DE LANGOSTINO RW CHUBU.mp4'
'PESCA EN RAWSO.mp4'
'Un dia de pesca merluza  Buque Paola.mp4'
'WhatsApp Video 2021-07-25 at 10.21.14 PM.mp4'
'WhatsApp Video 2021-07-25 at 9.24.42 PM.mp4'


In [4]:
INPUT_VIDEOS = [
    (
        'buque EL MARISCO II ,proceso de captura merluz.mp4',
        None,
    ),
    
    (
        'LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4',
        None,        
    ),
    
    (
        'Maniobra de pesca de merluza en mares argentino.mp4',
        None,
    ),
    
    ( 
        'maniobr.mp4',
        None
    ),
    
    (
        'Pesca artesanal de anchoita en mar del plat.mp4',
        None
    ),
    
    (
        'pesca de altura en el mar argentino,pesquero.mp4',
        None
    ),
    
    (
        'pesca de langostino,buque Alve.mp4',
        None
    ),
    
    (
        'Pesca de langostino en aguas nacionale.mp4',
        None
    ),
    
    (
        'pesca de langostino en rawson chubut con barco combenciona [camara-fija].mp4',
        None,
    ),
    
    (
        'PESCA DE LANGOSTINO RW CHUBU.mp4',
        None,
    ),
    
    (
        'PESCA EN RAWSO.mp4',
        None
    ),
    
    (
        'Un dia de pesca merluza  Buque Paola.mp4',
        None
    ),
    
    (
        'WhatsApp Video 2021-07-25 at 10.21.14 PM.mp4',
        None
    ),
    
    (
        'WhatsApp Video 2021-07-25 at 9.24.42 PM.mp4',
        None
    )
]

In [5]:
!ls {WORKSPACE_DATA_PATH}/output

job0.mp4


In [6]:
!ls {WORKSPACE_DATA_PATH}/models/yolov4-416-tf

assets	coco.names  saved_model.pb  variables


In [7]:
MODELS_PATH=WORKSPACE_DATA_PATH+"models/"
!ls {MODELS_PATH}

coco  kaggle_fisheries	kaggle-fisheries-yolo  README.md  yolov4-416-tf


In [9]:
output_path=WORKSPACE_DATA_PATH+"/output/"
dry_run=False

for i,v in enumerate(INPUT_VIDEOS[:1]):
    input_video_filename=v[0]
    output_video_filename=output_path+"job{:d}.mp4".format(i)
    
    params=v[1]
    detector_model_weights_filename=MODELS_PATH+"/yolov4-416-tf"
    detector_model_classes_filename=MODELS_PATH+"/yolov4-416-tf"+"/coco.names"
    
    print(input_video_filename)
    
    if not dry_run:
        context,pipeline=make_detection_pipeline(
            input_video_filename=INPUT_VIDEO_PATH+input_video_filename,
            output_video_filename=output_video_filename,
            detector_model_weights_filename=detector_model_weights_filename,
            detector_model_classes_filename=detector_model_classes_filename,
            start_frame=0,
            max_frames=1000
        )
        
        pipeline.execute()
        print("Tiempo total de ejecución [s]:", pipeline.get_total_execution_time())
        print("Video de salida guardado en {}".format(output_video_filename))
    
    

buque EL MARISCO II ,proceso de captura merluz.mp4


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 794.2511507660001
Video de salida guardado en ..//data//output/job0.mp4


In [11]:
import tensorflow as tf
print("GPUs disponibles: ", len(tf.config.list_physical_devices('GPU')))

GPUs disponibles:  0
